# Import libraries and load CSV-Files

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

df_hat = pd.read_csv("/Users/erikfriedrich/Downloads/Nat_Gas_Hat.csv")
df_fill = pd.read_csv("/Users/erikfriedrich/Downloads/Nat_Gas_Filled.csv")

# Building the Functions

In [55]:
# this function gets the prices for each specified date
# this will be used to calculate how much money will be made by buying and selling the security

def GetPrices(injection_dates, withdrawal_dates):
    
    cutoff_date = datetime.strptime("2024-09-01", "%Y-%m-%d") # cutoff date, that decides in which dataframe we'll search
    prices_dict = {} # create an empty dict to store the dates and prices 
    
    # get's the prices for each injection (buying)
    for i in injection_dates:
        
        # same logic as in the chapter before
        date = datetime.strptime(i, "%Y-%m-%d")
        formatted_date = date.strftime("%Y-%m-%d")
    
        if date > cutoff_date:
            price = float(df_hat["yhat"].loc[df_hat["ds"] == formatted_date]) 
        else:
            price = float(df_fill["y"].loc[df_fill["ds"] == formatted_date])
        
        # add new key to our dict, with negative price, because we're buying
        prices_dict[i] = -price
    
    # get's the prices at each withdrawal (selling)
    for j in withdrawal_dates:
        
        # same logic as in the chapter before
        date = datetime.strptime(j, "%Y-%m-%d")
        formatted_date = date.strftime("%Y-%m-%d")
    
        if date > cutoff_date:
            price = float(df_hat["yhat"].loc[df_hat["ds"] == formatted_date]) 
        else:
            price = float(df_fill["y"].loc[df_fill["ds"] == formatted_date])
        
        # positive price because we're selling
        prices_dict[j] = price
        
    return prices_dict

In [56]:
# this function calculates the dates that we've stored the Gas
# we need this to calculate how much money we have to spend on storage

# the function calculates the deltas between injection and withdrawal dates and adds them together

def DaysStored(injection_dates, withdrawal_dates):
    
    days = 0
    
    for i in range(len(injection_dates)):
        
        inj_date = datetime.strptime(injection_dates[i], "%Y-%m-%d")
        with_date = datetime.strptime(withdrawal_dates[i], "%Y-%m-%d")
    
        delta = with_date - inj_date
        days += delta.days
        
    return days

In [57]:
# this function returns the price of the contract

def PriceContract(injection_dates, withdrawal_dates, pumping_costs, max_store, storage_costs):
    
    # calculates the revenue we've made from buying and selling the security
    # assuming that we always buy the maximum amount that is possible
    prices = GetPrices(injection_dates, withdrawal_dates)
    rev = sum(prices.values()) * max_store
    
    # counts the times we've injection or withdrawan gas times the amount each time costs us
    pumping = len(prices) * pumping_costs
    
    # calculates the storage costs depending on the days stored times the rate for each day
    days = DaysStored(injection_dates, withdrawal_dates) 
    storage = days * storage_costs
    
    # calculates the value of the contract (revenue - costs)
    # and returns it
    value = rev - pumping - storage
    return value

# Sample Inputs

In [60]:
injection_dates = ["2020-10-1", "2024-12-1"]
withdrawal_dates = ["2024-12-1", "2025-04-15"]
pumping_costs = 1_000
max_store = 1_000
storage_costs = 1_000

PriceContract(injection_dates, withdrawal_dates, pumping_costs, max_store, storage_costs)

{'2020-10-1': -10.1,
 '2024-12-1': 12.805616985777249,
 '2025-04-15': 11.987705218882263}

# Limitations

We assume that the number of times we inject and withdraw gas are equal and we always start by injecting.
Meaning that we start without a position in gas and end without having a position.

We also assume that we always buy the maximum amount that is possible.
This constraint implies that there can't be two injection dates without a withdrawal date inbetween and vice versa.

Another constraint is, that there can't be buying and selling the same day. 
(Which is quite logical in this model, since the revenue from that is zero but it comes with injection and withdrawal costs, so it always has negative EV)